In [50]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sps
from scipy.stats import pareto
import math 
import statistics 

In [41]:
n = 100
teta = 5

In [42]:
x = pareto.rvs(b=teta-1, size=n)

In [43]:
teta_e = n/sum(np.log(x)) + 1

# Confidence interval

## Confidence interval for median

In [44]:
med_b1 = 2**(1/(teta_e-1)) - 1.96*2**(1/(teta_e-1))*np.log(2)/((teta_e-1)*n**0.5)
med_b2 = 2**(1/(teta_e-1)) + 1.96*2**(1/(teta_e-1))*np.log(2)/((teta_e-1)*n**0.5)

In [61]:
print(f'interval for median: ({med_b1}, {med_b2}) real median: {statistics.median(x)}')

interval for median: (1.1578788037710899, 1.2441020597153796) real median: 1.191368826035503


## Asymptotic for theta

In [64]:
as_b1 = teta_e - 1.96*(teta_e - 1)/(n**0.5)
as_b2 = teta_e + 1.96*(teta_e - 1)/(n**0.5)

In [66]:
print(f'interval for theta: ({as_b1}, {as_b2}) real theta: {teta}')

interval for theta: (4.042865145836736, 5.526451137339225) real theta: 5


In [71]:
delta = np.array([])
for i in range(0,1000):
    bootstrap = np.random.choice(x, size = x.shape[0]).round(2) 
    delta = np.append(delta, n/sum(np.log(bootstrap)) + 1 - teta_e)

delta.sort() 
b1_boot, b2_boot = teta_e - delta[974], teta_e - delta[24]

In [72]:
print(f'bootstrap interval for theta : ({b1_boot}, {b2_boot}) real theta: {teta}')

bootstrap interval for theta : (3.844506157711658, 5.461518453181821) real theta: 5
